In [3]:
import numpy as np  # this module handles arrays, but here we need it for its NaN value
import pandas as pd # this module contains a lot of tools for handling tabular data
from matplotlib import pyplot as plt
from salishsea_tools import evaltools as et
import datetime as dt
import os
import gsw
import pickle
import netCDF4 as nc
import cmocean
from scipy import stats as spst
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

## How CSV was created:

# Load csv:

In [2]:
df=pd.read_csv('/data/eolson/results/MEOPAR/oldDBs/HPLCPhyto.csv')

In [3]:
df=pd.read_csv('/data/eolson/results/MEOPAR/oldDBs/HPLCPhyto.csv',dtype={'dtUTC':str})
# convert dtUTC back to datetime
df['dtUTC']=[dt.datetime.strptime(ii,'%Y-%m-%d %H:%M:%S') for ii in df['dtUTC']]
df.head()

,dtUTC,Lat,Lon,Z,p,i,j,k,Sample_Number,Diatoms-1,...,Dictyochophytes,Raphidophytes,Cyanobacteria,TchlA (ug/L),Chlorophyll:Extracted [mg/m^3],Nitrate_plus_Nitrite [umol/L],Silicate [umol/L],Fluorescence:URU [mg/m^3],SA,CT
0,2015-04-01 23:42:17,48.65233,-123.50183,1.487321,1.5,207,354,1,9.0,23.580,...,0.000,0.163,0.000,26.162,29.36,4.63,19.43,NaN,27.295384,10.045218
1,2015-04-02 03:24:50,48.63017,-123.24284,2.082250,2.1,242,331,2,23.0,0.272,...,0.000,0.000,0.000,0.439,0.67,23.23,41.81,NaN,29.374328,9.201191
2,2015-04-02 15:23:46,48.49983,-124.73317,1.586497,1.6,7,413,1,37.0,2.033,...,0.005,0.023,0.004,3.129,3.43,5.70,9.85,NaN,29.643997,10.547005
3,2015-04-02 18:19:18,48.46900,-124.54850,1.883969,1.9,31,394,1,51.0,2.722,...,0.014,0.033,0.010,4.219,4.69,15.58,24.48,NaN,30.492932,9.644676
4,2015-04-02 23:03:35,48.30817,-124.06650,1.685682,1.7,82,328,1,64.0,3.341,...,0.002,0.022,0.000,6.385,7.15,1.17,3.43,NaN,29.434686,11.089943


## Match data

In [4]:
# path to model files:
PATH= '/results/SalishSea/nowcast-green.201812/'

# start and end dates for analysis:
start_date = dt.datetime(2015,4,1)
end_date = dt.datetime(2015,5,1)

# number of days per model file:
flen=1

# dictionary mapping desired model variables to the file types where they are found
filemap={'nitrate':'ptrc_T','silicon':'ptrc_T','ammonium':'ptrc_T','diatoms':'ptrc_T','ciliates':'ptrc_T',
         'flagellates':'ptrc_T','vosaline':'grid_T','votemper':'grid_T'}

# dictionary mapping model file types to their time resolution in hours (1 is hourly files, 24 is daily)
fdict={'ptrc_T':1,'grid_T':1}

# results format
# -- nowcast: files like 01jan15/SalishSea_1h_20150101_20150101_ptrc_T.nc
# -- long: files like SalishSea_1h_20150206_20150804_ptrc_T_20150427-20150506.nc, all in one directory
namfmt='nowcast'

data=et.matchData(df,filemap,fdict,start_date,end_date,namfmt,PATH,flen,preIndexed=True)

In [5]:
data

,dtUTC,Lat,Lon,Z,p,i,j,k,Sample_Number,Diatoms-1,...,SA,CT,mod_nitrate,mod_silicon,mod_ammonium,mod_diatoms,mod_ciliates,mod_flagellates,mod_vosaline,mod_votemper
0,2015-04-01 23:42:17,48.65233,-123.50183,1.487321,1.5,207,354,1,9.0,23.580,...,27.295384,10.045218,0.207268,18.074280,0.061200,9.364264,0.538119,1.354740,27.297085,10.330333
1,2015-04-02 03:24:50,48.63017,-123.24284,2.082250,2.1,242,331,2,23.0,0.272,...,29.374328,9.201191,16.684338,38.885910,1.431027,2.013328,0.262435,0.523047,29.845945,9.545719
2,2015-04-02 15:23:46,48.49983,-124.73317,1.586497,1.6,7,413,1,37.0,2.033,...,29.643997,10.547005,5.142227,13.671011,0.531737,0.624700,0.046742,0.196337,28.772097,11.000047
3,2015-04-02 18:19:18,48.46900,-124.54850,1.883969,1.9,31,394,1,51.0,2.722,...,30.492932,9.644676,7.909885,19.923824,0.697330,1.382296,0.200322,0.537118,29.407999,10.531830
4,2015-04-02 23:03:35,48.30817,-124.06650,1.685682,1.7,82,328,1,64.0,3.341,...,29.434686,11.089943,5.250195,17.457861,0.507802,4.250437,0.419683,0.868200,30.088562,10.704157
5,2015-04-03 02:09:56,48.26083,-123.72017,1.685690,1.7,127,294,1,76.0,5.729,...,30.092338,10.375860,16.168316,36.099716,1.318640,1.465542,0.312656,0.623337,30.610191,9.816114
6,2015-04-03 05:34:12,48.23283,-123.30100,1.586536,1.6,185,259,1,86.0,4.217,...,30.367032,10.207919,14.524487,35.114857,1.585027,1.287341,0.345707,0.630652,30.366905,9.777546
7,2015-04-03 08:46:05,48.26483,-123.16300,1.784847,1.8,209,255,1,96.0,4.665,...,30.455359,9.832826,12.663631,32.201111,1.489424,1.889705,0.363822,0.591890,30.294970,9.950639
8,2015-04-03 10:34:51,48.24317,-122.97517,1.983166,2.0,234,238,1,109.0,1.254,...,30.019539,9.968907,12.999703,32.490604,1.477581,1.741756,0.356779,0.576574,30.354095,9.955937
9,2015-04-03 11:55:44,48.38000,-123.04350,1.983141,2.0,240,269,1,120.0,0.290,...,30.199268,9.437491,16.255709,37.583473,1.669954,0.906741,0.279977,0.520383,30.367716,9.629019


In [4]:
test=et.loadDFOCTD()

In [5]:
test

,Year,Month,Day,Hour,Lat,Lon,Z,SA,CT,Fluor,dtUTC
0,2014.0,2.0,11.0,6.598056,48.618333,-123.248333,0.991551,31.189609,7.327741,0.567,2014-02-11 06:35:53
1,2014.0,2.0,11.0,6.598056,48.618333,-123.248333,1.983098,31.191168,7.327932,0.596,2014-02-11 06:35:53
2,2014.0,2.0,11.0,6.598056,48.618333,-123.248333,2.974640,31.191120,7.327442,0.612,2014-02-11 06:35:53
3,2014.0,2.0,11.0,6.598056,48.618333,-123.248333,3.966176,31.197000,7.329882,0.577,2014-02-11 06:35:53
4,2014.0,2.0,11.0,6.598056,48.618333,-123.248333,4.957709,31.201373,7.331038,0.576,2014-02-11 06:35:53
...,...,...,...,...,...,...,...,...,...,...,...
819620,2019.0,12.0,12.0,15.786111,49.322600,-124.001568,385.200000,31.512139,9.599687,NaN,2019-12-12 15:47:10
819621,2019.0,12.0,12.0,15.786111,49.322600,-124.001568,386.100000,31.504219,9.570673,NaN,2019-12-12 15:47:10
819622,2019.0,12.0,12.0,15.786111,49.322600,-124.001568,387.000000,31.503033,9.547747,NaN,2019-12-12 15:47:10
819623,2019.0,12.0,12.0,15.786111,49.322600,-124.001568,388.100000,31.500741,9.519935,NaN,2019-12-12 15:47:10
